In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install -q pmdarima


In [3]:
# Challenge 1: Time Series Demand Forecasting
# Predict daily product demand across stores using historical sales and inventory data

In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,StandardScaler

%matplotlib inline

ModuleNotFoundError: No module named 'tensorflow.keras'

In [ ]:
# Load the dataset
file_path = '../data/retail_store_inventory.csv'
df = pd.read_csv(file_path)

df.head()

In [ ]:
df.info()

In [ ]:
df['Category']

In [ ]:
#check null
print("\nMissing Values:")
print(df.isnull().sum())

In [ ]:
#Key Data Features

# Date : Daily records from [start_date] to [end_date].

# Store ID & Product ID : Unique identifiers for stores and products.

# Category : Product categories like Electronics, Clothing, Groceries, etc.

# Region : Geographic region of the store.

# Inventory Level : Stock available at the beginning of the day.

# Units Sold : Units sold during the day.

# Demand Forecast : Predicted demand based on past trends.

# Weather Condition : Daily weather impacting sales.

# Holiday/Promotion : Indicators for holidays or promotions.

In [ ]:
# dispay frequency count for categorical columns
categorical_columns=df.select_dtypes(include=['object']).columns
for col in categorical_columns:
    print(f"\nFrequency counts for {col}:")
    print(df[col].value_counts())
    print(f"Percentage distribution for {col}:")

In [ ]:
# Aggregate data by date

#convert data to datetime
df['Date']= pd.to_datetime(df['Date'])
daily_sales = df.groupby('Date')['Units Sold'].sum().reset_index()

# Plot daily sales
plt.figure(figsize=(12, 6))
plt.plot(daily_sales['Date'], daily_sales['Units Sold'])
plt.title('Time series plot of daily sales')
plt.xlabel('Date')
plt.ylabel('Units Sold')
plt.show()

In [ ]:
#Time Series Demand Forecasting

In [ ]:
df = df.sort_values('Date')
daily_sales = df.groupby('Date')['Units Sold'].sum().reset_index()
daily_sales.set_index('Date', inplace=True)
daily_sales.head(2)

In [ ]:
SeasonalityCounts = df['Seasonality'].value_counts().reset_index()
SeasonalityCounts

In [ ]:
#creating box plot for show the outliers


In [ ]:
# 1. Visualize data to check for outliers
# Plot boxplots for 'Inventory Level', 'Units Sold', and 'Price'
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
sns.boxplot(x=df['Inventory Level'])
plt.title('Inventory Level Outliers')

plt.subplot(1, 3, 2)
sns.boxplot(x=df['Units Sold'])
plt.title('Units Sold Outliers')

plt.subplot(1, 3, 3)
sns.boxplot(x=df['Price'])
plt.title('Price Outliers')

plt.tight_layout()
plt.show()

In [ ]:
#units sold contain outliers , now checking how much data counts are outliers
# Calculate Q1 (25th percentile) and Q3 (75th percentile)
Q1 = df['Units Sold'].quantile(0.25)
Q3 = df['Units Sold'].quantile(0.75)

# Calculate IQR
IQR = Q3 - Q1

# Define the bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
print(df.shape)
print(df[df['Units Sold'] > upper_bound].shape)
print(df[df['Units Sold'] < lower_bound].shape)

### above results are showing the outlier are in the upper_bound
##only and less count comparing to the orignal one so we can remove the outliers.

In [ ]:
# Detecting potential outliers using IQR (Interquartile Range)
Q1 = df[['Inventory Level', 'Units Sold', 'Price']].quantile(0.25)
Q3 = df[['Inventory Level', 'Units Sold', 'Price']].quantile(0.75)
IQR = Q3 - Q1
outliers = ((df[['Inventory Level', 'Units Sold', 'Price']] < (Q1 - 1.5 * IQR)) | 
            (df[['Inventory Level', 'Units Sold', 'Price']] > (Q3 + 1.5 * IQR)))
print("Potential Outliers Detected in units sold :")
print(outliers.sum())

In [ ]:
#Detect and remove outliers using IQR (Interquartile Range)
Q1 = df[['Inventory Level', 'Units Sold', 'Price']].quantile(0.25)
Q3 = df[['Inventory Level', 'Units Sold', 'Price']].quantile(0.75)
IQR = Q3 - Q1

# Define the outlier condition
outliers_condition = ((df[['Inventory Level', 'Units Sold', 'Price']] <= (Q1 - 1.5 * IQR)) | 
                      (df[['Inventory Level', 'Units Sold', 'Price']] >= (Q3 + 1.5 * IQR)))

# Remove outliers 
#This filters the original DataFrame df by selecting only rows where any(axis=1) is False (i.e rows that do not contain any outliers).
#The tilda (~) negates the condition, so rows with no outliers are kept, and rows with outliers are removed.

df_cleaned = df[~outliers_condition.any(axis=1)]

# any(axis=1): Ensures that we consider rows with outliers 
#in any of the columns. If a row has an outlier in at least one column, it will be flagged as an outlier.

# Display the shape before and after removal
print(f"Original dataset shape: {df.shape}")
print(f"Cleaned dataset shape: {df_cleaned.shape}")
df=df_cleaned

In [ ]:
# Feature engineering: add day of week and month
daily_sales['DayOfWeek'] = daily_sales.index.dayofweek
daily_sales['Month'] = daily_sales.index.month
daily_sales.head(2)

In [ ]:
# Normalize the data
scaler = MinMaxScaler()
sales_scaled = scaler.fit_transform(daily_sales)

In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(x='Date', y='Inventory Level', data=df)
plt.title('Inventory Level Over Time')
plt.show()

In [ ]:
plt.figure(figsize=(8, 4))
sns.scatterplot(x='Units Sold', y='Inventory Level', data=df)
plt.title('Inventory Level vs Units Sold')
plt.show()

In [ ]:
# Calculate overstock frequency 
avg_daily_sales = df.groupby('Date')['Units Sold'].mean().mean()
overstock_freq = (df['Inventory Level'] > 2 * avg_daily_sales).mean()
print(f"Overstock frequency: {overstock_freq:.2%}")

In [ ]:
df.head(3)

In [ ]:
# Sort the DataFrame by date and store/product ID to ensure correct lag and rolling calculations
df = df.sort_values(['Store ID', 'Product ID', 'Date'])

# Create lag features
for lag in [1, 7]:
    df[f'Sales_Lag_{lag}'] = df.groupby(['Store ID', 'Product ID'])['Units Sold'].shift(lag)
    
# Sales_Lag_1: how many units were sold 1 day before
# Sales_Lag_7: how many units were sold 7 days before

In [ ]:
df.head(3)
#This tells the model: “Yesterday we sold 20 units, and 7 days ago we sold 30 units
#Capture short-term demand trends (1-day lag = immediate trend)
#Capture weekly seasonality (7-day lag = weekday pattern)

In [ ]:
# Create rolling mean features
for window in [7, 14]:
    df[f'Sales_RollingMean_{window}'] = df.groupby(['Store ID', 'Product ID'])['Units Sold'].transform(lambda x: x.rolling(window=window, min_periods=1).mean())
    
#Sales_RollingMean_7: The average units sold over the past 7 days for that store product
#Sales_RollingMean_14: Same over the past 14 days.

In [ ]:
df.head(3)

In [ ]:
# Add day of week and month
df['DayOfWeek'] = df['Date'].dt.dayofweek
df['Month'] = df['Date'].dt.month

# Handle missing values
df = df.dropna()

# Print the shape of the DataFrame before and after dropping NA values
print(f"Shape before dropping NA: {df.shape}")
print(f"Shape after dropping NA: {df.shape}")

In [ ]:
df = df.dropna()


In [ ]:
# Check the created features
print(df[['Units Sold'] + [f'Sales_Lag_{lag}' for lag in [1, 7]] + [f'Sales_RollingMean_{window}' for window in [7, 14]] + ['DayOfWeek', 'Month']].head())

In [ ]:
df.head(3)

In [ ]:
#change the string values in numeric because ML model not be traind on string values
le=LabelEncoder()
df['Store ID']=le.fit_transform(df['Store ID'])
df['Product ID']=le.fit_transform(df['Product ID'])
df['Category']=le.fit_transform(df['Category'])
df['Region']=le.fit_transform(df['Region'])
df['Weather Condition']=le.fit_transform(df['Weather Condition'])
df['Seasonality']=le.fit_transform(df['Seasonality'])


In [ ]:
df.head(2)

In [ ]:
df.columns

In [ ]:
print("\nMissing Values:")
print(df.isnull().sum())

In [ ]:
df.drop(columns=['Store ID','Product ID','Date'],axis=1,inplace=True)

In [ ]:
df.head(5)

In [ ]:
corr_matrix = df.corr()

# Create a heatmap
plt.figure(figsize=(9, 7))  # Set figure size
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f", cbar=True)

 
plt.title("Correlation Heatmap")
plt.show()

In [ ]:
df.info()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import numpy as np

# Define features and target
# features = ['Sales_Lag_1', 'Sales_Lag_7', 'Sales_RollingMean_7', 'DayOfWeek', 'Month', 'Price', 'Discount']
# X = df[]
# y = df['']


# X=df.drop('Units Sold',axis=1)
# y=df['Units Sold']

# # Train-test split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Train Random Forest model
# rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
# rf_model.fit(X_train, y_train)

# # Make predictions
# y_pred = rf_model.predict(X_test)

# # Evaluation metrics
# mse = mean_squared_error(y_test, y_pred)
# mape = mean_absolute_percentage_error(y_test, y_pred) * 100  # Convert to percentage

# print(f"Mean Squared Error (MSE): {mse:.2f}")
# print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")


In [ ]:
wape = np.sum(np.abs(y_test - y_pred)) / np.sum(np.abs(y_test)) * 100
print(f"WAPE: {wape:.2f}%")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Get feature importances and sort them
importances = rf_model.feature_importances_
feature_names = X.columns
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Plot
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importance_df, palette='viridis')
plt.title('Feature Importance from Random Forest')
plt.xlabel('Importance Score')
plt.ylabel('Features')
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score
import matplotlib.pyplot as plt

# Calculate additional metrics
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae:.2f}")
print(f"R-squared Score: {r2:.2f}")

# Visualize predictions vs actual
plt.figure(figsize=(6, 4))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel("Actual ")
plt.ylabel("Predicted ")
plt.title("Actual vs Predicted ")
plt.show()

# # Feature importance
# importances = rf_model.feature_importances_
# feature_imp = pd.DataFrame(sorted(zip(importances, features)), columns=['Y','X'])

# plt.figure(figsize=(6, 4))
# sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
# plt.title('Random Forest Features Importance')
# plt.tight_layout()
# plt.show()

# Time Series Demand Forecasting


In [ ]:
# Objective:
# Build and evaluate models to predict daily product
#demand using time series forecasting techniques

## Outcome
# Generate forecasts for daily product demand
# and assess the performance of the ARIMA model. Use this model to understand trends and improve inventory planning

In [ ]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df[:2]

In [ ]:
df.columns

In [ ]:
df.drop(columns=['Store ID','Product ID'],axis=1,inplace=True)

In [ ]:

# 1. Data Preparation
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

In [ ]:
# Aggregate the data by day and sum the 'Units Sold' for daily demand
daily_demand = df.resample('D')['Units Sold'].sum()
daily_demand

In [ ]:
#ARIMA models only require a single time-series column 


In [ ]:
# ADF Test for Stationarity
# Objective:
# To check if the time series data is stationary using
#the Augmented Dickey-Fuller (ADF) test.

In [ ]:
# 2. Check for Stationarity
# Perform ADF Test
def adf_test(series):
    result = adfuller(series)
    print(f"ADF Statistic: {result[0]}")
    print(f"p-value: {result[1]}")

adf_test(daily_demand)
# If p-value > 0.05  Non-stationary  Apply differencing.

# If p-value < 0.05  Stationary  Good to proceed.

#Stationarity: A time series is considered stationary if its statistical properties, 
#like mean, variance, and autocorrelation, remain constant over time.
#In simple terms, the pattern in the data does not change, making it easier to model and forecast.


# ADF Statistic: A negative value that indicates how strong the trend is in the time series.

#p-value: Indicates whether we can reject the null hypothesis that the series is 
#non-stationary. If the p-value is below 0.05, we can reject the null hypothesis and
#consider the series stationary.

In [ ]:
# 3. Model Building (ARIMA)
# Split the data into training and testing sets
train_size = int(len(daily_demand) * 0.8)
train, test = daily_demand[:train_size], daily_demand[train_size:]

In [ ]:
# Fit the ARIMA model (p=1, d=0, q=1 as a starting point, can be tuned)
model = ARIMA(train, order=(1, 0, 1))
model_fit = model.fit()
# (1, 0, 1)	Basic model with autoreg & noise
# (2, 0, 1)	Add more lag
# (1, 0, 2)	More MA
# (0, 0, 1)	MA-only model
# (1, 0, 0)	AR-only model

In [ ]:
# 4. Forecasting
forecast = model_fit.forecast(steps=len(test))

In [ ]:
# 5. Model Evaluation
# Calculate RMSE
rmse = sqrt(mean_squared_error(test, forecast))
print(f"RMSE: {rmse}")

In [ ]:
# Plot actual vs forecasted values
plt.figure(figsize=(12, 6))
plt.plot(test.index, test, label='Actual')
plt.plot(test.index, forecast, label='Forecasted', color='red')
plt.title('ARIMA Model - Daily Demand Forecasting')
plt.xlabel('Date')
plt.ylabel('Units Sold')
plt.legend()
plt.show()

In [ ]:
###### trying different #### FuRTHER IMproving

# Data Prep	Clean datetime, handle missing values, preserve full date info
# Stationarity	Automate ADF test + differencing suggestion
# Order Selection	Add ACF/PACF plots 
# Model Fit	Train ARIMA with optimal (p,d,q) and clean diagnostics
# Forecast	Plot forecast + confidence intervals
# Evaluation	RMSE, MAE, and residual diagnostics
# Structure	Wrap into functions for reusability

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df.sort_values('Date', inplace=True)
df.set_index('Date', inplace=True)

In [ ]:
# Target Variable
ts = df['Units Sold'] 

In [ ]:
# ================= 2. STATIONARITY CHECK =================
def test_stationarity(timeseries):
    result = adfuller(timeseries.dropna())
    print(f'ADF Statistic: {result[0]}')
    print(f'p-value: {result[1]}')
    return result[1] < 0.05

is_stationary = test_stationarity(ts)

# ================= 3. DIFFERENCING IF NEEDED =================
d = 0
if not is_stationary:
    ts = ts.diff().dropna()
    d = 1
    print("Series differenced (d=1)")


In [ ]:

# ================= 4. PLOT ACF / PACF =================
fig, ax = plt.subplots(1, 2, figsize=(12, 4))
plot_acf(ts, ax=ax[0])
plot_pacf(ts, ax=ax[1])
plt.suptitle("ACF and PACF")
plt.show()

# ================= 5. MODEL FIT =================
# You can change (p,d,q) after looking at plots
order = (1, d, 1)
model = ARIMA(df['Units Sold'], order=order)
model_fit = model.fit()
print(model_fit.summary())

In [ ]:
# ================= 6. FORECAST =================
n_steps = 30
forecast = model_fit.get_forecast(steps=n_steps)
forecast_mean = forecast.predicted_mean
conf_int = forecast.conf_int()

In [ ]:

# ================= 7. PLOT =================
plt.figure(figsize=(12, 5))
plt.plot(df['Units Sold'], label='Observed')
plt.plot(forecast_mean.index, forecast_mean, color='red', label='Forecast')
plt.fill_between(conf_int.index,
                 conf_int.iloc[:, 0],
                 conf_int.iloc[:, 1], color='pink', alpha=0.3)
plt.title("Forecast: Units Sold")
plt.legend()
plt.show()

In [ ]:
# ================= 8. METRICS =================
# Compare last known with forecast start for RMSE
actual = df['Units Sold'][-n_steps:]
predicted = model_fit.predict(start=len(df)-n_steps, end=len(df)-1)

rmse = sqrt(mean_squared_error(actual, predicted))
mae = mean_absolute_error(actual, predicted)
print(f'RMSE: {rmse:.2f}, MAE: {mae:.2f}')

# ================= 9. DIAGNOSTICS =================
model_fit.plot_diagnostics(figsize=(12, 8))
plt.show()

In [ ]:
#compare them to your average Units Sold:

mean_sales = df['Units Sold'].mean()
print(f"Mean Units Sold: {mean_sales:.2f}")
print(f"RMSE as % of mean: {115.01 / mean_sales * 100:.2f}%")
print(f"MAE as % of mean: {102.16 / mean_sales * 100:.2f}%")


In [ ]:
# ARIMAX Implementation # Add external regressors 

In [ ]:
# Load the dataset
file_path = '/kaggle/input/retail-store-inventory-forecasting-dataset/retail_store_inventory.csv'
df = pd.read_csv(file_path)

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df.sort_values('Date', inplace=True)
df.set_index('Date', inplace=True)

In [ ]:
df.columns

In [ ]:
le=LabelEncoder()
df['Store ID']=le.fit_transform(df['Store ID'])
df['Product ID']=le.fit_transform(df['Product ID'])
df['Category']=le.fit_transform(df['Category'])
df['Region']=le.fit_transform(df['Region'])
df['Weather Condition']=le.fit_transform(df['Weather Condition'])
df['Seasonality']=le.fit_transform(df['Seasonality'])

In [ ]:
#  Select Target + Exogenous Variables
target = df['Units Sold']
exog_vars = ['Price', 'Discount', 'Inventory Level', 
             'Demand Forecast', 'Competitor Pricing', 'Seasonality','Weather Condition','Region']
exog = df[exog_vars].fillna(method='bfill')  # Fill missing values

In [ ]:
n_steps = 30

train_y = target[:-n_steps]
train_exog = exog[:-n_steps]

test_y = target[-n_steps:]
test_exog = exog[-n_steps:]


In [ ]:
model = ARIMA(train_y, exog=train_exog, order=(1, 0, 1))
model_fit = model.fit()
print(model_fit.summary())


In [ ]:
forecast_result = model_fit.get_forecast(steps=n_steps, exog=test_exog)
forecast_mean = forecast_result.predicted_mean
conf_int = forecast_result.conf_int()


In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(target, label='Observed')
plt.plot(forecast_mean.index, forecast_mean, color='red', label='ARIMAX Forecast')
plt.fill_between(conf_int.index,
                 conf_int.iloc[:, 0],
                 conf_int.iloc[:, 1], color='pink', alpha=0.3)
plt.legend()
plt.title("ARIMAX Forecast: Units Sold")
plt.show()


In [ ]:
rmse = sqrt(mean_squared_error(test_y, forecast_mean))
mae = mean_absolute_error(test_y, forecast_mean)

# Optional: MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    nonzero = y_true != 0
    return np.mean(np.abs((y_true[nonzero] - y_pred[nonzero]) / y_true[nonzero])) * 100

mape = mean_absolute_percentage_error(test_y, forecast_mean)

print(f'RMSE: {rmse:.2f}')
print(f'MAE: {mae:.2f}')
print(f'MAPE: {mape:.2f}%')


TREE BASED MODEL FORECASTING

In [ ]:
############ Trying XGboost  ################

df.head(3)

In [ ]:
features = ['Category', 'Region',
       'Inventory Level', 'Units Ordered', 'Demand Forecast',
       'Price', 'Discount', 'Weather Condition', 'Holiday/Promotion',
       'Competitor Pricing', 'Seasonality', 'Sales_Lag_1', 'Sales_Lag_7',
       'Sales_RollingMean_7', 'Sales_RollingMean_14', 'DayOfWeek', 'Month']
target = 'Units Sold'


In [ ]:
df = df.dropna()  # Drop initial NaNs due to lag


In [ ]:
from sklearn.model_selection import train_test_split

X = df[features]
y = df[target]

# Keep last 30 days as test
train_size = int(len(df) * 0.9)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


In [ ]:
from xgboost import XGBRegressor

model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=4, random_state=42)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

mape = mean_absolute_percentage_error(y_test, y_pred)

print(f'RMSE: {rmse:.2f}')
print(f'MAE: {mae:.2f}')
print(f'MAPE: {mape:.2f}%')


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))
plt.plot(y_test.index, y_test, label='Actual Units Sold')
plt.plot(y_test.index, y_pred, label='XGBoost Prediction', linestyle='--')
plt.legend()
plt.title("XGBoost Forecast vs Actual")
plt.show()


In [ ]:
from xgboost import plot_importance

plt.figure(figsize=(12, 6))
plot_importance(model, max_num_features=5, importance_type='gain', 
                title='Top 10 Feature Importances (XGBoost)', 
                show_values=True)
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()


In [ ]:
#### hyperparmeter tuning xgboost

In [ ]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1]
}


In [ ]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from xgboost import XGBRegressor

tscv = TimeSeriesSplit(n_splits=3)

xgb = XGBRegressor(random_state=42)
grid_search = GridSearchCV(estimator=xgb,
                           param_grid=param_grid,
                           cv=tscv,
                           scoring='neg_mean_squared_error',
                           verbose=1,
                           n_jobs=-1)


In [ ]:
grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_


In [ ]:
y_pred = best_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

print(f'Tuned RMSE: {rmse:.2f}')
print(f'Tuned MAE: {mae:.2f}')
print(f'Tuned MAPE: {mape:.2f}%')


| Stage         | RMSE   | MAE    | MAPE   | Notes                          |
| ------------- | ------ | ------ | ------ | ------------------------------ |
| **ARIMA**     | 115.01 | 102.16 | 84.28% | Basic univariate, poor fit     |
| **ARIMAX**    | 8.68   | 7.04   | 43.97% | Stronger, but MAPE still high  |
| **XGBoost**   | 8.36   | 7.15   | 18.51% | Huge MAPE drop, stable metrics |
| **Tuned XGB** | 8.33   | 7.12   | 18.51% | *Polished performance*         |


In [ ]:
# ****Month Forecast with Tuned XGBoost ###########


In [ ]:
# Load the dataset
file_path = '/kaggle/input/retail-store-inventory-forecasting-dataset/retail_store_inventory.csv'
df = pd.read_csv(file_path)

df.head()

In [ ]:
df.head()

In [ ]:
# # === STEP 1: PREPROCESSING & FEATURE CREATION ===
# df['Date'] = pd.to_datetime(df['Date'])
# df = df.sort_values('Date')

# # Temporal features
# df['DayOfWeek'] = df['Date'].dt.dayofweek
# df['Month'] = df['Date'].dt.month


In [ ]:
df['Date']

In [ ]:
# # Add temporal features
# future_df['DayOfWeek'] = future_df['Date'].dt.dayofweek
# future_df['Month'] = future_df['Date'].dt.month


In [ ]:
# future_predictions = []

# last_known = df.copy()

# for date in future_df['Date']:
#     temp_row = {
#         'Date': date,
#         'DayOfWeek': date.dayofweek,
#         'Month': date.month,
#         # Fill in from last known or rolling predictions
#         'Sales_Lag_1': last_known['Units Sold'].iloc[-1],
#         'Sales_Lag_7': last_known['Units Sold'].iloc[-7] if len(last_known) >= 7 else last_known['Units Sold'].mean(),
#         'Sales_RollingMean_7': last_known['Units Sold'].rolling(7).mean().iloc[-1],
#         'Sales_RollingMean_14': last_known['Units Sold'].rolling(14).mean().iloc[-1],
#         # Add other static or known values
#         'Price': last_known['Price'].iloc[-1],
#         'Discount': last_known['Discount'].iloc[-1],
#         # etc.
#     }

In [ ]:
   # # Convert to DataFrame for prediction
   #  X_future = pd.DataFrame([temp_row])
   #  y_pred = best_model.predict(X_future)[0]
    
   #  temp_row['Units Sold'] = y_pred
   #  last_known = last_known.append(temp_row, ignore_index=True)
   #  future_predictions.append({'Date': date, 'Forecasted Units Sold': y_pred})

In [ ]:
# forecast_df = pd.DataFrame(future_predictions)

# import matplotlib.pyplot as plt

# plt.figure(figsize=(14, 6))
# plt.plot(df['Date'], df['Units Sold'], label='Historical Sales')
# plt.plot(forecast_df['Date'], forecast_df['Forecasted Units Sold'], label='4-Month Forecast', color='green')
# plt.title("XGBoost Forecast for Next 4 Months")
# plt.xlabel("Date")
# plt.ylabel("Units Sold")
# plt.legend()
# plt.grid(True)
# plt.tight_layout()
# plt.show()
